In [9]:
import yfinance as yf
from auxillary import *
from indicators import *
from datetime import date
from processData import processData

In [10]:
stocks = []
print("Enter the ticker symbols of interest (enter spacebar to exit)")
s = "  "
while s != " ":
    s = input("Ticker: ")
    if s == " " and len(stocks) == 0:
        print("Please enter a ticker symbol")
        s = "  "
        continue
    elif s == " ":
        break
    if s.upper() not in stocks:
        stocks.append(s.upper())
    

n = int(input("How many trading days in the future would you like to predict? "))



Enter the ticker symbols of interest (enter spacebar to exit)


In [11]:
general, target, cluster, pc = processData(stocks, n)